<a href="https://colab.research.google.com/github/arashHarirpoosh/FaceRecognition/blob/main/RetinaFaceTracker_retinaface_pytorch_lib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import lib's

In [1]:
!pip install pytube
!pip install --upgrade youtube-dl
!pip install 'h5py==2.10.0' --force-reinstall
# !pip install retinaface
!pip install deepface
!pip install -U retinaface_pytorch > /dev/null

     |████████████████████████████████| 56 kB 2.4 MB/s 
     |████████████████████████████████| 1.9 MB 5.5 MB/s 
     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |████████████████████████████████| 15.7 MB 198 kB/s 
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires h5py~=3.1.0, but you have h5py 2.10.0 which is incompatible.
tensorflow 2.6.0 requires numpy~=1.19.2, but you have numpy 1.21.2 which

     |████████████████████████████████| 61 kB 3.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.3 MB 10.9 MB/s 
     |████████████████████████████████| 4.0 MB 45.4 MB/s 
     |████████████████████████████████| 14.8 MB 184 kB/s 
  Created wheel for gdown: filename=gdown-4.0.2-py3-none-any.whl size=10613 sha256=5d2a186f772f88b4da023aadc82944d74f0789c93147d9180c325624774af793
  Stored in directory: /root/.cache/pip/wheels/75/20/08/4d138cf98df8fe9e14e369c7de79e01c5cabca55b0f58cef3a
Successfully built gdown
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.2
    Uninstalling numpy-1.21.2:
      Successfully uninstalled numpy-1.21.2
  Attempting uninstall: h5py
    Found existing installation: h5py 2.1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.20 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.4.1 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.28.0 which is incompatible.


In [ ]:
# !rm ./*jpg
# %rm -rf drive

rm: cannot remove './*jpg': No such file or directory


In [2]:
from pytube import YouTube
from pytube.exceptions import RegexMatchError
import youtube_dl
import dlib
# from retinaface import RetinaFace
from deepface import DeepFace
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import cv2
import os
import threading
import time
import torch
import gc
from tqdm import tqdm
import time
import numpy as np
import json
import pandas as pd

from retinaface.pre_trained_models import get_model
from retinaface.utils import vis_annotations


Directory  /root /.deepface created
Directory  /root /.deepface/weights created


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Codes

In [4]:
def check_tracker_quality(face_trackers, base_image, frame_num, quality_thresh_hold=7):
    # Update all the trackers and remove the ones for which the update
    # indicated the quality was not good enough
    fids_to_delete = []
    for fid in face_trackers.keys():
        trackingQuality = face_trackers[fid].update(base_image)

        # If the tracking quality is good enough, we must delete
        # this tracker
        if trackingQuality < quality_thresh_hold:
            fids_to_delete.append(fid)

    for fid in fids_to_delete:
        print("Removing tracker " + str(fid) + " from list of trackers")
        del face_trackers[fid]
        df.loc[df['faceID'] == float(fid), 'LastFrame'] = frame_num
    return face_trackers


def create_new_tracker(current_face_id, frame, loc):
    print("Creating new tracker " + str(current_face_id))

    x, y, w, h = loc
    # Create and store the tracker
    tracker = dlib.correlation_tracker()
    tracker.start_track(frame,
                        dlib.rectangle(x - int(w / 4),
                                       y - int(h / 4),
                                       x + w + int(w / 4),
                                       y + h + int(h / 4)))
    return tracker

In [5]:
def do_recognize_person(face_names, fid, face_img):
    verified_identity = None
    try:
        face_img_bgr = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
        df_fr = DeepFace.find(face_img_bgr,
                           db_path="/content/FaceDataBase",
                           enforce_detection=False,
                           detector_backend='retinaface',
                           model_name='ArcFace',
                           model=model,
                           distance_metric="euclidean_l2")
        print(df_fr)
        print()
        # filtered_faced = df_fr[df_fr['ArcFace_euclidean_l2'] <= 1]
        closest_identity = df_fr.loc[df_fr['ArcFace_euclidean_l2'].idxmin()]
        print(closest_identity['identity'])
        # verified_identity = closest_identity['identity'].split('/')[-2].split('\\')[1]
        verified_identity = closest_identity['identity'].split('/')[-2]

        print()
        print(verified_identity)
        del df_fr
        gc.collect()
    except (ValueError, cv2.error):
        gc.collect()
        print('No Face Detected')
        # all_faces = RetinaFace.detect_faces(data)
        # print(all_faces)
        # cv2.imshow('face', face_img)
        # cv2.waitKey()

    num_of_zeros = 3 - len(str(fid))
    if verified_identity is None:
        face_names[fid] = {
            'name': '{person_id}'.format(person_id=num_of_zeros * '0' + str(fid)),
            'detected': False
        }
    else:
        face_names[fid] = {
            'name': verified_identity,
            'dir': '{person_id}'.format(person_id=verified_identity + '/' + num_of_zeros * '0' + str(fid)),
            'detected': True
        }


def check_existence_of_new_faces(result_list, face_trackers,
                                 face_names, frame, current_face_id, frame_num):
    verified_trackers = []
    for result in result_list:
      if len(result['bbox']) > 3:
        x, y, x1, y1 = result['bbox']
        w, h = abs(x1 - x), abs(y1 - y)

        # Calculate the center point
        x_bar = (x + x1) / 2
        y_bar = (y + y1) / 2

        # Variable holding information which faceid we
        # matched with
        matched_fid = None

        # Now loop over all the trackers and check if the
        # center point of the face is within the box of a
        # tracker
        for fid in face_trackers.keys():
            tracked_position = face_trackers[fid].get_position()
            t_x = int(tracked_position.left())
            t_y = int(tracked_position.top())
            t_w = int(tracked_position.width())
            t_h = int(tracked_position.height())

            # calculate the center point
            t_x_bar = t_x + 0.5 * t_w
            t_y_bar = t_y + 0.5 * t_h

            # check if the center point of the face is within the
            # rectangle of a tracker region. Also, the center point
            # of the tracker region must be within the region
            # detected as a face. If both of these conditions hold
            # we have a match
            if ((t_x <= x_bar <= (t_x + t_w)) and
                    (t_y <= y_bar <= (t_y + t_h)) and
                    (x <= t_x_bar <= (x + w)) and
                    (y <= t_y_bar <= (y + h))):
                matched_fid = fid
                verified_trackers.append(fid)

        if matched_fid is None:
            loc = [x, y, w, h]
            tracker = create_new_tracker(current_face_id, frame, loc)
            df.loc[df.shape[0] + 1] = [current_face_id, None, frame_num, None, x, y, w, h, None]

            face_trackers[current_face_id] = tracker
            verified_trackers.append(current_face_id)
            # Start a new thread that is used to simulate
            # face recognition. This is not yet implemented in this
            # version :)
            # t = threading.Thread(target=do_recognize_person,
            #                      args=(face_names, current_face_id))
            # t.start()
            # Increase the currentFaceID counter
            current_face_id += 1

    # Remove the trackers that are not in the second detection
    all_fids = face_trackers.copy().keys()
    for fid in all_fids:
        if fid not in verified_trackers:
            print("Removing tracker " + str(fid) +
                  " from list of trackers because of not appearing in the second detection")
            del face_trackers[fid]
            df.loc[df['faceID'] == float(fid), 'LastFrame'] = frame_num
    del all_fids
            
    return current_face_id, face_trackers

In [6]:
def plot_image_with_boxes(frame, face_trackers, face_name, num_of_digits,
                          frame_number, output_file_name, face_shapes=None, save_image=True):
    face_img = frame.copy()
    for fid in face_trackers.keys():
        tracked_position = face_trackers[fid].get_position()
        t_x = int(tracked_position.left())
        t_y = int(tracked_position.top())
        t_w = int(tracked_position.width())
        t_h = int(tracked_position.height())

        # face = frame[t_y:t_y + t_h, t_x:t_x + t_w]
        face = face_img[t_y:t_y + t_h, t_x:t_x + t_w]
        if face_shapes is not None:
            face = cv2.resize(face, face_shapes, fx=0, fy=0, interpolation=cv2.INTER_CUBIC)

        if fid in face_name.keys():
            box_name = face_name[fid]['name']

        else:
            box_name = 'Detecting...'
            start = time.time()
            do_recognize_person(face_name, fid, face)
            elapsed_verification_time.append(time.time() - start)
        if save_image:
            if face_name[fid]['detected']:
                df.loc[df['faceID'] == float(fid), 'Identity'] = face_name[fid]['name']
                directory = '/'.join(output_file_name) + '/labeled/' + face_name[fid]['dir']
                id_person = face_name[fid]['dir'].split('/')[1]
            else:
                directory = '/'.join(output_file_name) + '/unlabeled/' + face_name[fid]['name']
                id_person = face_name[fid]['name']
            # directory = output_file_name + '\\unlabeled\\' + face_name[fid]
            if not os.path.exists(directory):
                os.makedirs(directory)

            num_of_zeros = num_of_digits - len(str(frame_number))
            img_name = directory + '/{source}_{n_video}_{frameNumber}_{id_p}_{x}_{y}_{h}_{w}.jpg'.format(
                source=output_file_name[-2], n_video=output_file_name[-1]
                , frameNumber=num_of_zeros * '0' + str(frame_number)
                , id_p=id_person, x=t_x, y=t_y, h=t_h, w=t_w)

            try:
                cv2.imwrite(img_name, face)
            except cv2.error:
                pass

        cv2.rectangle(frame, (t_x, t_y),
                      (t_x + t_w, t_y + t_h),
                      (0, 0, 255), 2)
        cv2.putText(frame, box_name, (int(t_x + t_w / 2), int(t_y)),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (204, 204, 0), 2)

In [7]:

def face_tracking(path, output_file_name, detection_quality_tsh=7, save_image=True, show_result=True,
                  face_detection_tsh=10, resize_faces_shape=None, desired_time=None, start_frame=0, current_face_id=1):
   
    if show_result:
        cv2.namedWindow("Face-Tracking", cv2.WINDOW_AUTOSIZE)  ###make window for image or video

        # Start the window thread for the two windows we are using
        cv2.startWindowThread()

    # Used as counter variable
    frame_counter = 0
    # current_face_id = 1

    # Variables holding the correlation trackers and the name per faceid
    face_trackers = {}
    face_names = {}

    # Check to see if previous run is available continue the last run
    framePath = '/'.join(output_file_name)
    runInfoAddr = framePath + '/runInfo.csv'
    if os.path.isfile(runInfoAddr):
      # df = pd.read_csv(runInfoAddr)
      frame_counter = int(df.iloc[0]['frameCounter'])
      start_frame = frame_counter
      current_face_id = int(df['faceID'].loc[df['faceID'].idxmax()]) + 1
      print(current_face_id)
      for index, row in df.iterrows():
        if not pd.notnull(row['LastFrame']):
          vidObj = cv2.VideoCapture(path)
          vidObj.set(1, start_frame - 1)
          success, frame = vidObj.read()
          del vidObj
          fid = int(row['faceID'])
          loc = [int(row['loc_x']), int(row['loc_y']), int(row['loc_w']), int(row['loc_h'])]
          face_trackers[fid] = create_new_tracker(fid, frame, loc)
          num_of_zeros = 3 - len(str(fid))

          verified_identity = row['Identity']
          if not pd.notnull(verified_identity):
              face_names[row['faceID']] = {
                  'name': '{person_id}'.format(person_id=num_of_zeros * '0' + str(fid)),
                  'detected': False
              }
          else:
              face_names[fid] = {
                  'name': verified_identity,
                  'dir': '{person_id}'.format(person_id=verified_identity + '/' + num_of_zeros * '0' + str(fid)),
                  'detected': True
              }

    # Path to video file
    start = time.time()
    vidObj = cv2.VideoCapture(path)
    # Get frame/second of the video
    frames_per_second = int(vidObj.get(cv2.CAP_PROP_FPS))
    length = vidObj.get(cv2.CAP_PROP_FRAME_COUNT)
    
    # vidObj.set(start_frame, length)
    vidObj.set(1, start_frame)
    end = time.time()
    print('DecodeTime:', end - start)
    print('Frames/second:', frames_per_second)
    print('Total numer of frames:', length)

    if desired_time is None:
        frame_threshold = int(vidObj.get(cv2.CAP_PROP_FRAME_COUNT))
    else:
        frame_threshold = int(desired_time * 60 * frames_per_second)

    number_of_all_frames_digits = len(str(frame_threshold))


    # while vidObj.isOpened() and frame_counter < frame_threshold:
    for frame_counter in tqdm(range(start_frame+1, frame_threshold+1), desc='Loading'):
        # vidObj object calls read
        # function extract frames
        start = time.time()
        success, image = vidObj.read()
        elapsed_decode_time.append(time.time() - start)

        # if frame_counter > start_frame:
        #   print(success)

        # if success and frame_counter > start_frame :
        if success:
            # resize_frame = cv2.resize(image, (1280, 720), fx=0, fy=0, interpolation=cv2.INTER_CUBIC)
            start = time.time()
            resize_frame = image
            face_trackers = check_tracker_quality(face_trackers, resize_frame,
                                                  quality_thresh_hold=detection_quality_tsh, frame_num=frame_counter)
            elapsed_checkquality_time.append(time.time() - start)

            # Every 10 frames, we will have to determine which faces#### is that 10 frames or face_detection_tsh times
            # are present in the frame
            if (frame_counter % face_detection_tsh) == 0:
                start = time.time()
                # all_faces = RetinaFace.detect_faces(resize_frame)
                all_faces = detection_model.predict_jsons(resize_frame)
                # print(all_faces)
                elapsed_detection_time.append(time.time() - start)

                if len(all_faces) > 0:
                    start = time.time()
                    current_face_id, face_trackers = check_existence_of_new_faces(all_faces, face_trackers,
                                                                                  face_names,
                                                                                  resize_frame, current_face_id, frame_counter)
                    elapsed_checkForNewTracker_time.append(time.time() - start)

            # frame_counter += 1
            start = time.time()
            plot_image_with_boxes(resize_frame, face_trackers, face_names, number_of_all_frames_digits,
                                  start_frame + frame_counter, output_file_name, face_shapes=resize_faces_shape, save_image=save_image)
            elapsed_faceExtraction_time.append(time.time() - start)
          
            if os.path.isdir(framePath):
              # df.iloc[0]['frameCounter'] = frame_counter
              df.iloc[0, -1] = frame_counter              
              df.to_csv(runInfoAddr, encoding='utf-8')
            if show_result:
                cv2.imshow('Face-Tracking', resize_frame)
                # Press Q on keyboard to  exit
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    break

        elif (not success) or (frame_counter == frame_threshold):
            vidObj.release()

    if show_result:
        # Destroy any OpenCV windows and exit the application
        cv2.destroyAllWindows()

In [8]:
def loadFromYT(link, name, output):
  # link = 'https://www.youtube.com/watch?v=o8tXSiKW-h8'
  try:
    yt = YouTube(link)
    stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
    stream.download(filename=name, output_path=output)
  except RegexMatchError:
    # 137: 1080p
    # 136: 720p
    # 135: 480p
    # 134: 360p
    # 133: 240
    ydl_opts = {
      # 'format': 'bestvideo/best',
      'format':'136',
      'outtmpl': f'{output}/{name}',
      }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([link])

# Download The Face Dataset

In [9]:
# import shutil

# shutil.rmtree('frames')

In [10]:

# !apt install subversion
!npx degit https://github.com/arashHarirpoosh/FaceRecognition/FaceDataBase FaceDataBase --force


npx: installed 1 in 2.321s
> cloned arashHarirpoosh/FaceRecognition#HEAD to FaceDataBase


#Main

In [ ]:
if __name__ == '__main__':
    print('dlib version:', dlib.__version__)

    elapsed_decode_time = []
    elapsed_detection_time = []
    elapsed_checkquality_time = []
    elapsed_checkForNewTracker_time = []
    elapsed_faceExtraction_time = []
    elapsed_verification_time = []


    model = DeepFace.build_model('ArcFace')
    
    detection_model = get_model("resnet50_2020-07-20", max_size=2048)
    detection_model.eval()

    base_addr = 'Videos'

    # num_of_sources = len(os.listdir(base_addr))
    # source_digits = len(str(num_of_sources))
    source_digits = 3

    # selected_channels = ['1', '2']
    selected_channels = ['1']
    selected_videos = [3]
    # 1:CNN, 2:BBC
    # videos = {
    #     '1':
    #           {
    #               # '1': 'https://www.youtube.com/watch?v=CmCp6Z54GYU',
    #               # '2':'https://youtu.be/pJlnxbO5N2g',
    #               # '3':'https://youtu.be/e0sGuSaGvV0',
    #               # '4': 'https://youtu.be/ZO_UqxwasO4',
    #               # '5':'https://youtu.be/fx2W1Jh4kA0',
    #               '6': 'https://www.youtube.com/watch?v=6y877A9IA4U',
    #               # '7':'https://www.youtube.com/watch?v=Rli9ZRvemh0',
    #               # '8':'https://www.youtube.com/watch?v=cQAH7d-k068',
    #               # '9':'https://www.youtube.com/watch?v=dG7ya0Iq8U0',
    #               # '10':'https://www.youtube.com/watch?v=Slu4WdZKq0A',
    #               # '11':'https://www.youtube.com/watch?v=nt5iBZCgqvs',
    #               # '12':'https://www.youtube.com/watch?v=X59GyNfVFeM',


    #           }
    #     # '2':
    #     #       {
    #     #           '1':'https://www.youtube.com/watch?v=sXQLYOHihUg',
                  
    #     #       }
    # }
    
    with open('drive/MyDrive/videos.json') as json_file:
      videos = json.load(json_file)

    for channels, videos in videos.items():
        file_digits = 4
        # root = 'Videos/{c}'.format(c=channels)
        root = 'drive/MyDrive/Videos/{c}'.format(c=channels)
        for file_number, link in videos.items():
          if int(file_number) in selected_videos:
            file_name = file_number+'.mp4'
            if not os.path.isfile(root + '/' + file_name):
              loadFromYT(link=link, name=file_name, output=root)

            root_list = root.split('/')
            # root_list[-2] = 'drive/MyDrive/Frames'
            root_list[-2] = 'Frames'
            source_number = root_list[-1]
            root_list[-1] = (source_digits - len(source_number)) * '0' + source_number
            root_list.append((file_digits - len(file_number)) * '0' + file_number)
            p = f'{root}/{file_name}'

            runInfoAddr = '/'.join(root_list) + '/runInfo.csv'
            columns = [
                         'faceID',
                         'Identity',
                         'StartFrame',
                         'LastFrame',
                         'loc_x',
                         'loc_y',
                         'loc_w',
                         'loc_h',
                         'frameCounter'
              ]
            if os.path.isfile(runInfoAddr):
              df = pd.read_csv(runInfoAddr, usecols=columns)
              # dtypes = {
              #            'faceID':int,
              #            'Identity':str,
              #            'StartFrame':int,
              #            'LastFrame':int,
              #            'loc_x':int,
              #            'loc_y':int,
              #            'loc_w':int,
              #            'loc_h':int,
              #            'frameCounter':int
              # }
              # for col, dtype in dtypes.items():
              #   df[col] = df[col].astype(dtype)
            else:

              df = pd.DataFrame(columns=columns)
            
            print(p)
            print(root_list)

            start_time = time.time()
            face_tracking(
                path=p,
                output_file_name=root_list,
                detection_quality_tsh=7,
                face_detection_tsh=20,
                # resize_faces_shape=(300, 400),
                resize_faces_shape=None,
                save_image=True,
                show_result=False,
                # desired_time=1,
                # start_frame=8954,
                # current_face_id = 155
            )

            elapsed_time = time.time() - start_time
            time_report = f'Finished in {elapsed_time} seconds, or {elapsed_time / 60} minutes'
            print(time_report)
            file = open(r"{folder}/time.txt".format(folder='/'.join(root_list)), "w+")
            file.write(time_report)
            file.close()


dlib version: 19.18.0
drive/MyDrive/Videos/1/3.mp4
['drive', 'MyDrive', 'Frames', '001', '0003']
DecodeTime: 0.0554046630859375
Frames/second: 29
Total numer of frames: 18979.0


Loading:   0%|          | 16/18979 [00:11<02:01, 155.56it/s]

Creating new tracker 1
Creating new tracker 2
Creating new tracker 3
Already built model is passed
retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /root/.deepface/weights/retinaface.h5

  0%|          | 0.00/119M [00:00<?, ?B/s]
  9%|▉         | 10.5M/119M [00:00<00:01, 104MB/s]
 18%|█▊        | 21.0M/119M [00:00<00:01, 62.2MB/s]
 32%|███▏      | 37.7M/119M [00:00<00:00, 81.3MB/s]
 46%|████▌     | 54.5M/119M [00:00<00:00, 104MB/s] 
 56%|█████▌    | 66.1M/119M [00:00<00:00, 86.8MB/s]
 64%|██████▍   | 76.0M/119M [00:01<00:00, 67.1MB/s]
 80%|███████▉  | 94.4M/119M [00:01<00:00, 71.6MB/s]
 87%|████████▋ | 103M/119M [00:01<00:00, 74.3MB/s] 
100%|██████████| 119M/119M [00:01<00:00, 68.1MB/s]


Representations stored in  /content/FaceDataBase / representations_arcface.pkl  file. Please delete this file when you add new identities in your database.
find function lasts  99.89003705978394  seconds
Empty DataFrame
Columns: [identity, ArcFace_euclidean_l2]
Index: []

No Face Detected
Already built model is passed
There are  43  representations found in  representations_arcface.pkl
find function lasts  2.6472487449645996  seconds
Empty DataFrame
Columns: [identity, ArcFace_euclidean_l2]
Index: []

No Face Detected
Already built model is passed
There are  43  representations found in  representations_arcface.pkl


...

Loading:   0%|          | 66/18979 [03:04<7:50:38,  1.49s/it] 

Removing tracker 2 from list of trackers


Loading:   0%|          | 77/18979 [03:21<1:37:49,  3.22it/s]

Creating new tracker 4
Already built model is passed
There are  43  representations found in  representations_arcface.pkl
find function lasts  1.888580083847046  seconds
Empty DataFrame
Columns: [identity, ArcFace_euclidean_l2]
Index: []

No Face Detected


Loading:   1%|          | 179/18979 [05:23<1:03:54,  4.90it/s]

In [ ]:
    print(f'Decode Time:\n Mean: {np.mean(elapsed_decode_time)} seconds,\n All: {np.sum(elapsed_decode_time)/60.0} mins')
    print(f'Detection Time:\n Mean: {np.mean(elapsed_detection_time)} seconds,\n All: {np.sum(elapsed_detection_time)/60.0} mins')
    print(f'Time For Checking Quality Of Trackers:\n Mean: {np.mean(elapsed_checkquality_time)} seconds,\n All: {np.sum(elapsed_checkquality_time)/60.0} mins')
    print(f'Time For Checking For New Trackers:\n Mean: {np.mean(elapsed_checkForNewTracker_time)} seconds,\n All: {np.sum(elapsed_checkForNewTracker_time)/60.0} mins')
    print(f'Time For Labelling And Saving Faces:\n Mean: {np.mean(elapsed_faceExtraction_time)} seconds,\n All: {np.sum(elapsed_faceExtraction_time)/60.0} mins')

Decode Time:
 Mean: 0.002061899503072103 seconds,
 All: 0.05979508558909098 mins
Detection Time:
 Mean: 24.527051681695983 seconds,
 All: 17.57772037188212 mins
Time For Checking Quality Of Trackers:
 Mean: 0.029441905432733995 seconds,
 All: 0.8538152575492859 mins
Time For Checking For New Trackers:
 Mean: 0.007171736207119254 seconds,
 All: 0.005139744281768799 mins
Time For Labelling And Saving Faces:
 Mean: 0.014532128969828287 seconds,
 All: 0.4214317401250203 mins
